# Report


---



## 1.   로딩 사이트 변경

LLM 이 참고하는 웹페이지를 스파르타 홈페이지로 바꾸었다. 이 과정에서 KoreanWebLoader 가 필요했다. 이는 홈페이지가 한글로 이루어져 있었고, 인코딩 방식에 변경이 필요했기 때문이다. 이것보다 더 간단한 방법으로 인코딩을 바꿀 방법이 있을걸로 보여서, 더 알아볼 필요가 있다.

## 2.   웹 검색 기능 추가

DuckDuckGo 무료 API 를 이용해서 검색 기능을 추가했다. 검색 결과 얻은 내용 역시 답변에 반영한다.


In [ ]:
import os
import bs4
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.docstore.document import Document

from bs4 import BeautifulSoup
import requests

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key="")

# DuckDuckGo 기반 검색
search_tool = DuckDuckGoSearchRun()

def duckduckgo_retriever(question: str) -> list[str]:
    result = search_tool.run(question)
    return [result]

# Web 문서 로드
class KoreanWebLoader(WebBaseLoader):
    def _scrape(self, url, bs_kwargs=None):
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "html.parser")
        return soup


    def _parse(self, html):
        soup = BeautifulSoup(html, "html.parser")
        elements = soup.find_all(class_="editedContent")
        texts = [e.get_text(separator="\n", strip=True) for e in elements if e.get_text(strip=True)]
        return [Document(page_content="\n\n".join(texts))]

loader = KoreanWebLoader(["https://spartacodingclub.kr/blog/all-in-challenge_winner"])
docs = loader.load()

# 벡터 분할 및 임베딩
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key="")
)
chroma_retriever = vectorstore.as_retriever()

# 최종 retriever: DuckDuckGo + Chroma 결과 합치기
def hybrid_retriever(question: str) -> dict:
    ddg_result = duckduckgo_retriever(question)
    chroma_result = chroma_retriever.invoke(question)
    chroma_text = "\n\n".join(doc.page_content for doc in chroma_result)

    return {
        "context_ddg": "\n\n".join(ddg_result),
        "context_chroma": chroma_text,
        "question": question
    }

# 프롬프트
prompt_template = PromptTemplate.from_template("""
너는 정보 요약 전문가야. 아래 두 가지 출처(실시간 웹 검색, 특정 블로그)에서 추출한 내용을 참고해서 질문에 답변해줘.
추측하지 말고, 아래 문맥을 기반으로 정확하게 대답해.

[실시간 DuckDuckGo 검색 결과]
{context_ddg}

[블로그 분석 결과]
{context_chroma}

[질문]
{question}
""")

# 전체 파이프라인
rag_chain = (
    RunnableLambda(hybrid_retriever)
    | prompt_template
    | llm
    | StrOutputParser()
)

# 실행
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."

retrieved = hybrid_retriever(user_msg)

print("[DuckDuckGo 검색 결과 요약]:\n")
print(retrieved["context_ddg"])
print("\n" + "="*100 + "\n")

print("[Chroma (블로그) 추출 문서 내용]:\n")
print(retrieved["context_chroma"])
print("\n" + "="*100 + "\n")


response = rag_chain.invoke(user_msg)

print("응답:\n")
print(response)


USER_AGENT environment variable not set, consider setting it to identify your requests.


[DuckDuckGo 검색 결과 요약]:

💡 코딩 공모전에서 만든 다양한 서비스를 만나보고 싶다면? 다양한 서비스와 기발한 아이디어가 모인 곳에 초대합니다. 참가자들의 문제 해결방법이 궁금하시다면 지금 바로 'All-in 공모전'에서 만나보세요! 공모전 마감일이 5월 13일까지였고, 저희는 3월 학기 초에 이야기를 했어서 2달 정도의 시간을 가지고 준비를 . 시작했습니다! ... 저는 작년 수상작들을 많이 분석해 보면서 캠페인의 차별점이나 문제점, 인사이트를 찾으려고 노력했습니다 ... 대홍 공모전 꿀팁 3가지를 말씀드려볼게요! 1. 간단해라. 2. 수상작을 보라. 3. 철저하게 조사하라. 저렇게 총 3가지로 말씀드릴 수 있는데요 인턴, 신입, 대외활동 등의 합격 후기와 공모전, 취준 정보들을 공유해보세요. ... 유튜브, pdf, 웹사이트등 다양한 자료를 링크만 첨부 하면 요약해주기 때문에 과제하는 시간을 줄일 수 있어요! 2. ai 사용 후, 걸리지 않는 방법이 있나요? 사실 ai를 사용했다면, 100% ... 코딩 경진대회 에서 초등중등부는 스크래치와 엔트리를 이용한 블록코딩을 고등대학부는 c, c++, 자바, 파이썬 등 텍스트 코딩 역량평가를 진행하는데요. 전국 예선과 서울 본선이 있습니다. ... 4차 산업 아이디어 공모전. 주최 : 미래와 소프트웨어 재단 / (예상 ...


[Chroma (블로그) 추출 문서 내용]:

스파르타코딩클럽 | 블로그포인트로딩중쿠폰내 강의실국비 신청 내역수강권증명서숙제 피드백계정로그아웃 스파르타 소식'AII-in 코딩 공모전’ 수상작을 소개합니다조회수  816·6분 분량2024. 9. 3.코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 